# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisée en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par **RFE** (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_ess = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span><ul class="toc-item"><li><span><a href="#En-conservant-les-valeurs-par-défaut-des-hyperparamères" data-toc-modified-id="En-conservant-les-valeurs-par-défaut-des-hyperparamères-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>En conservant les valeurs par défaut des hyperparamères</a></span></li><li><span><a href="#En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille" data-toc-modified-id="En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille</a></span></li></ul></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Modèle final</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Régression SVM linéaire
- Régression SVM avec noyau
- Régression kNN
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XGBoost
- Perceptron multi-couches

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

#### En conservant les valeurs par défaut des hyperparamères

In [5]:
from functions.ml_modeling import get_models
from functions.ml_modeling import compare_models

# Create models
default_models = get_models(X_train, y_train)

# Compare models
compare_models(X_train, y_train, default_models)

,RMSE,RMSE_std,R2,R2_std
Ridge,1.225298,0.731001,-0.221627,1.574230
Lasso,1.302335,0.024078,-0.001867,0.002141
ElasticNet,1.281978,0.023693,0.029077,0.016423
LinearSVR,1.294355,0.708138,-0.304865,1.530210
SVR,0.836307,0.040984,0.586477,0.031824
KNeighborsRegressor,1.008789,0.038625,0.398683,0.032113
DecisionTreeRegressor,0.845726,0.026259,0.576822,0.028541
RandomForestRegressor,0.643409,0.040500,0.754909,0.027334
GradientBoostingRegressor,0.635189,0.033111,0.761350,0.020930
XGBRegressor,0.670338,0.035690,0.734257,0.022718


#### En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille

In [6]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.955646,0.066553,0.457714,0.079133
Lasso,1.072417,0.355106,0.239687,0.579399
ElasticNet,0.977885,0.108398,0.427271,0.139759
LinearSVR,1.208167,0.302127,0.078588,0.491968
SVR,0.779613,0.036656,0.640436,0.029816
KNeighborsRegressor,0.984303,0.021795,0.427676,0.011266
DecisionTreeRegressor,0.735019,0.044614,0.680169,0.033964
RandomForestRegressor,0.639299,0.040270,0.758054,0.026444
GradientBoostingRegressor,0.635693,0.034508,0.760961,0.021675
XGBRegressor,0.670338,0.035690,0.734257,0.022718


### Recherche des variables les plus significatives

Une grande qualité des forêts aléatoires est qu'elles permettent de mesurer facilement l'importance relative des variables. C'est donc tout naturellement que nous allons les utiliser pour sélectionner les variables les plus pertinentes.

Pour cela nous allons utiliser la classe **SelectFromModel** de *sklearn.feature_selection*. Cette classe permet de sélectionner toutes les variables dont l'importance (mesurée pour un modèle donné, donc ici d'une forêt aléatoire) est supérieure à un certain seuil, seuil qui est donné en paramètre. Nous utiliserons une recherche par grille pour trouver le seuil optimal.

Ensuite nous ne conserverons que les variables qui sont significatives et réentrainerons les modèles avec ces variables.

In [7]:
from functions.ml_modeling import select_features

# Get the mask of the features to be selected
forest_reg = [m for m in models if type(m).__name__=='RandomForestRegressor'][0]
mask = select_features(X_train, y_train, forest_reg)

# Use the mask to select the significant features only
X_train_select = X_train[:, mask].copy()
n_features_select = mask.sum()
print('{} features have been selected'.format(n_features_select))

33 features have been selected


Nous observons que les meilleurs résultats sont obtenus lorsque nous conservons 33 variables sur les 54. Par la suite, nous entrainerons nos modèles uniquement sur ces 33 variables.

In [8]:
# Create models
models = get_models(X_train_select, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train_select, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.958956,0.066370,0.453953,0.079585
Lasso,1.068229,0.347054,0.248536,0.562026
ElasticNet,0.977489,0.106056,0.428042,0.136566
LinearSVR,1.151436,0.322719,0.150025,0.542822
SVR,0.758725,0.041410,0.658995,0.036287
KNeighborsRegressor,0.971631,0.024978,0.442298,0.015908
DecisionTreeRegressor,0.732818,0.045665,0.681981,0.035304
RandomForestRegressor,0.638334,0.042324,0.758735,0.027743
GradientBoostingRegressor,0.635402,0.035625,0.761132,0.022787
XGBRegressor,0.661081,0.039784,0.741222,0.027768


### Sélection des cinq meilleurs modèles

In [9]:
# Get a score based on both RMSE and R2 scores
df['RMSE_minmax'] = (df['RMSE']-df['RMSE'].min())/(df['RMSE'].max()-df['RMSE'].min())
df['R2_minmax'] = (df['R2']-df['R2'].min())/(df['R2'].max()-df['R2'].min())
df['Score'] = df['R2_minmax'] - df['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df['Score'].nlargest(5).index
best_models = [m for m in models if type(m).__name__ in best_models_name]
print("The top five models are:")
for m in best_models_name:
    print(m)

The top five models are:
GradientBoostingRegressor
RandomForestRegressor
XGBRegressor
DecisionTreeRegressor
SVR


### Modèle final

Maintenant que nous avons réentrainé les modèles les plus prometteurs en ne conservant que les variables significatives, nous allons pouvoir obtenir le modèle finale.

Pour ce modèle finale, nous allons reprendre l'astuce des modèles ensemblistes, nous allons chercher à obtenir le meilleur des modèles sélectionnés en les empilant de manière à former un nouveau modèle plus performant. Pour cela, nous utiliserons la classe **StackingRegressor** de *sklearn.ensemble*.

In [10]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
best_models.append(stack_reg)
df = compare_models(X_train_select, y_train, best_models)
df

,RMSE,RMSE_std,R2,R2_std
SVR,0.758725,0.041410,0.658995,0.036287
DecisionTreeRegressor,0.732818,0.045665,0.681981,0.035304
RandomForestRegressor,0.638334,0.042324,0.758735,0.027743
GradientBoostingRegressor,0.635150,0.034999,0.761333,0.022369
XGBRegressor,0.661081,0.039784,0.741222,0.027768
StackingRegressor,0.620081,0.039212,0.772403,0.024693


In [11]:
models_stacking_name = ['Ridge',
                        'SVR',
                        'KNeighborsRegressor',
                        'RandomForestRegressor',
                        'GradientBoostingRegressor',
                        'XGBRegressor']
models_stacking = [m for m in models if type(m).__name__ in models_stacking_name]

# Get a stacking ensemble of models
estimators = []
for m in models_stacking:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
models_stacking.append(stack_reg)
df = compare_models(X_train_select, y_train, models_stacking)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.958956,0.066370,0.453953,0.079585
SVR,0.758725,0.041410,0.658995,0.036287
KNeighborsRegressor,0.971631,0.024978,0.442298,0.015908
RandomForestRegressor,0.638334,0.042324,0.758735,0.027743
GradientBoostingRegressor,0.634565,0.035077,0.761746,0.022737
XGBRegressor,0.661081,0.039784,0.741222,0.027768
StackingRegressor,0.615262,0.040455,0.775950,0.024908
